## pull in open ai key

In [6]:
# pull open ai key in
creds_file_path = '../openai.key'

try:
    with open(creds_file_path, 'r') as file:
        lines = file.readlines()
        openai_key = lines[0].strip()
except FileNotFoundError:
    print(f"The file {creds_file_path} does not exist.")
except IndexError:
    print("The credentials file does not contain enough lines for username and password.")
except Exception as e:
    print(f"An error occurred: {e}")

## Set up crew AI against a scrum team.

In [10]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import BaseTool
import os
search_tool = DuckDuckGoSearchRun()


llm_openai = ChatOpenAI(
    openai_api_key=openai_key,
    model_name="gpt-4o-mini"
)

class SaveStringToFileTool(BaseTool):
    name = "save_string_to_file"
    description = "Saves a multi-line string to a file."

    def _run(self, string_to_save: str, file_path: str) -> str:
        try:
            with open(file_path, 'w') as file:
                file.write(string_to_save)
            return f"Successfully saved the string to {file_path}."
        except Exception as e:
            return f"Failed to save the string: {str(e)}"

    async def _arun(self, string_to_save: str, file_path: str) -> str:
        return self._run(string_to_save, file_path)

save_string_to_specified_directory = SaveStringToFileTool()

manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    verbose=True,
    llm=llm_openai
)

# Create a researcher agent
researcher = Agent(
  role='Senior Researcher',
  goal='Collect data from function calling, searches and API calls',
  backstory='Help the IT Product Manager by collecting data',
  verbose=True,
  tools=[search_tool],
  allow_delegation=False,
  llm=llm_openai
)

outbound_product_manager = Agent(
  role='Outbound IT Product Manager',
  goal='Discover Key Insights',
  backstory='You are able to find key insights from the data you are given, \
  and using the knowledge that is already available to you as an LLM. Once your \
  insights have been critiqued by the insight critique agent and updated based \
  on that critique, you need to save your findings in a markdown file \
  in the ./doc directory using the tool provided', 
  tools=[save_string_to_specified_directory],
  verbose=True,
  allow_delegation=False,
  llm=llm_openai
)

insight_critique = Agent(
  role='Insights Critique',
  goal='Look for errors and gaps in the output from the outbound product manager',
  backstory='You are very detailed oriented person, and are responsible for \
  critiquing the output from the product manager  and returning \
  feedback to them',
  verbose=True,
  allow_delegation=False,
  llm=llm_openai
)


# inbound_product_manager = Agent(
#   role='Inbound IT Product Manager',
#   goal='Create detailed step by step plans',  
#   backstory='You are very detailed oriented person, and take the instructions \
#   provided to you by the outbound product manager, and come up with detailed \
#   and step by step plans for the software developer to code or for the \
#   technical writer to document. You also review the test code written by the \
#   software enginer under test and the actual functional code written by the \
#   software engineer to ensure it matches the requirements that you wrote',
#   verbose=True,
#   allow_delegation=False,
#   llm=llm_openai
# )


# critique = Agent(
#   role='Critique',
#   goal='Look for errors in the detailed plan and even take a contrarian perspective to enhance it',
#   backstory='You are very detailed oriented person, and are responsible for \
#   critiquing the detailed plans made by inbound IT product manager and returning \
#   feedback to them',
#   verbose=True,
#   allow_delegation=False,
#   llm=llm_openai
# )

# software_engineer_under_test = Agent(
#   role='test engineer',
#   goal='Write unit, functional and regression tests which capture the essence of \
#   the detailed plan provided by the product manager',
#   backstory='you are an experienced and competent Python developer who writes \
#   well documented and working software code which passes all the tests written \
#   by the software engineer under test. You save the code with descriptive file names \
#   under the test/ directory',
#   tools=[save_string_to_specified_directory],
#   allow_code_execution=True
#   verbose=True,
#   allow_delegation=False,
#   llm=llm_openai
# )

# software_engineer = Agent(
#   role='software engineer',
#   goal='Develop functional and well documented code which passes all tests',
#   backstory='you are an experienced and competent Python developer who writes \
#   well documented and working software code which passes all the tests written \
#   by the software engineer under test. You save the code with descriptive filenames \
#   under the src/ directory',
#   tools=[save_string_to_specified_directory],
#   allow_code_execution=True
#   verbose=True,
#   allow_delegation=False,
#   llm=llm_openai
# )

# tech_writer = Agent(
#   role='technical writer',
#   goal='writing comprehensive and easy to understand documentation',
#   backstory='you are an experienced technical writer who is able \
#   to write detailed technical documentation or come up with presentation \
#   slide concepts which expresses they why, the value, and the necessary \
#   details to the audience. You save the documentation with a descriptive file name \
#   in markdown format in the ./doc/ directory',
#   verbose=True,
#   tools=[save_string_to_specified_directory],
#   allow_delegation=False,
#   llm=llm_openai
# )

# Tasks
#description='Identify the documentation to write useful functions that can interact with salesforce products, snowflake, informatica IDMC, and airflow',:
research_task = Task(
  description='Identify all the kind of questions that can be asked in a medical school interview, \
  which includes a multi-panel interview of multiple interviewers and one interviewees, \
  group interview of 1 interviewer multiple interviewees, and a group activity as well as the best answers \
  that maximize the chance of admission',
  agent=researcher,
  expected_output='a bullet list of all questions and their answers'
)

#description='Do an industry analysis and a SWOT analysis to find out how the solution is built outside, and identify all salient functional and non functional requirements',
insights_task = Task(
  description='Compile and prioritize these questions and the best \
  possible answers which bring the maximum opportunity to pass the interview. \
  The interviewee is shy and introverted but incredibly smart, so explain the \
  answers in this context',
  agent=outbound_product_manager,
  expected_output='a bullet list of all questions and their answers'
)

# planner_task = Task(
#   description='Create a detailed step by step plan based on the requirements \
#   provided by the outbound product manager',
#   agent=inbound_product_manager,
#   expected_output='a detailed step by step plan, with the final updated plan sent to tech writer to document'
# )

insight_critique_task = Task(
  description='critically evaluate each detail created by the product manager, \
  to make the output more robust and aligned to the original requiremennts provided \
  to the outbound product manager.',
  agent=insight_critique,
  expected_output='thorough corrections to output of the product manager and bulletted reasons why,'  
)

# critique_task = Task(
#   description='critically evaluate each step created by the inbound product manager, \
#   to make the plan more robust and aligned to the original requirementns provided \
#   by the outbound product manager. Additionally, and only if the project involves coding, \
#   critique the test plan created by software_engineer_under_test \
#   to ensure it abides by the requirements, and critique the tests coded by the software_engineer_under_test to \
#   to ensure they cover the test plan',
#   agent=critique,
#   expected_output='thorough corrections to the plan or the tests, and reasons why,'  
# )

# test_writing_task = Task(
#   description='Using the test driven development concept, write a test plan, and functional unit tests \
#   based which fulfil the requirements',
#   agent=software_engineer_under_test,
#   expected_output='test plan with detailed test cases, and corresponding unit tests coded, \
#   with the test plan sent to tech writer to document'
# )

# software_development_task = Task(
#   description='Write solid software which aligns to the requirements',
#   agent=software_engineer,
#   expected_output='well documented, and python code, written using object oriented style, \
#   which aligns to the requirements and which passes 80% of the unit tests'
# )

documentation_writing_task = Task(
  description='Write excellent, descriptive, detailed, and well flowing documentation in markdown formwat',
  agent=tech_writer,
  expected_output='all information sent to it documented in markdown with examples, \
  with the documents saved in the ./doc/ directory'
)

# Instantiate your crew
tech_crew = Crew(
  # agents=[researcher, outbound_product_manager, inbound_product_manager, 
  #         critique, software_engineer_under_test, 
  #         software_engineer, tech_writer],
  #tasks=[research_task, insights_task, planner_task, critique_task, test_writing_task, software_development_task, documentation_writing_task],

  agents=[researcher, outbound_product_manager, insight_critique],    
  tasks=[research_task, insights_task, insight_critique_task],
  manager_agent=manager,
  process=Process.hierarchical
  #process=Process.sequential  # Tasks will be executed one after the other
)


2024-09-02 20:20:28,477 - 8553303872 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [11]:
# Begin the task execution
result = tech_crew.kickoff()
print(result)



> Entering new CrewAgentExecutor chain...
I need to gather comprehensive information on the types of questions that can be asked in various formats of medical school interviews, including multi-panel interviews, group interviews, and group activities. Additionally, I need to find the best responses that can enhance the chances of admission. Since this is a complex task requiring expertise in medical school admissions, I will delegate this work to the Senior Researcher.

Action: Delegate work to coworker
Action Input: {"task":"Identify all the kinds of questions that can be asked in a medical school interview, including multi-panel interviews (multiple interviewers and one interviewee), group interviews (one interviewer and multiple interviewees), and group activities. Additionally, provide the best answers that maximize the chance of admission.","context":"The task involves researching and compiling a comprehensive list of potential interview questions specific to medical school admi